# Recommendation System for Movies in Pytorch using Stack Auto Encoder

#### Part 1: Data Preprocessing

In [1]:
# importing the Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# importing the dataset
movies = pd.read_csv('dataset\ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('dataset\ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('dataset\ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
# Preparing the training set and the test set
training_set = pd.read_csv('dataset\ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('dataset\ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [4]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [5]:
# converting the data into array with users in lines and movies in column
def convert(data):
    new_data = []
    for id_users in range(1, nb_users +1):
        id_movies = data[:,1][data[:,0] == id_users ]
        id_ratings = data[:,2][data[:,0] == id_users ]
        ratings = np.zeros(nb_movies)
        ratings[id_movies -1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [6]:
training_set = convert(training_set)
test_set = convert(test_set)

In [7]:
# convert the data into torch tensor
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

#### Part 2: Building the Auto Encoder

In [8]:
# Creating the architecture of the Stack Auto Encoder Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

#### Part 3: Training and Testing Auto Encoder

In [9]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 1.7716305255771234
epoch: 2 loss: 1.0967801084957054
epoch: 3 loss: 1.0534703672820198
epoch: 4 loss: 1.0382487616912517
epoch: 5 loss: 1.0310048144190709
epoch: 6 loss: 1.0264577108558546
epoch: 7 loss: 1.023866060156085
epoch: 8 loss: 1.0218215750076067
epoch: 9 loss: 1.0206260127129247
epoch: 10 loss: 1.0196243124424016
epoch: 11 loss: 1.0188193786177713
epoch: 12 loss: 1.0184607558843952
epoch: 13 loss: 1.0177125568089451
epoch: 14 loss: 1.0174462516876264
epoch: 15 loss: 1.0170087667233798
epoch: 16 loss: 1.016699957587575
epoch: 17 loss: 1.016782234251401
epoch: 18 loss: 1.0166936734035497
epoch: 19 loss: 1.0163385860225136
epoch: 20 loss: 1.0160661608217583
epoch: 21 loss: 1.016066423249105
epoch: 22 loss: 1.0159179550592885
epoch: 23 loss: 1.0159481700236892
epoch: 24 loss: 1.015820553539149
epoch: 25 loss: 1.0153496082356233
epoch: 26 loss: 1.0156935108095215
epoch: 27 loss: 1.0154045487002659
epoch: 28 loss: 1.0148520554117353
epoch: 29 loss: 1.014002339456173


In [12]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9478524766918428
